# Solving integral equations

More specifically, we only consider solving *linear Fredholm* integral equations. 

Fredholm equations involve integrals of the unknown function over a fixed size interval.


## Fredholm equation of the first kind


$$
\int_a^b \! K(x, t)\, \color{blue}{f(t)} \,dt = g(x)
$$

Here $a$ and $b$ are fixed constants, $\color{blue}{f(x)}$ is the unknown real function of a real variable, $g(x)$ is the known function, the *right-hand side* (свободный член), and $K(x, t)$ is also a known function of two real variables, the *kernel* (ядро).

Loosely speaking, *a FE of the first kind only contains the unknown function under the integral sign.*

Note the similarity of a FE to a linear algebra problem $\mathbf{A} \mathbf{x} = \mathbf{b}$, with a matrix $\mathbf{A}$ and vectors $\mathbf{x}$ and $\mathbf{b}$. Here instead of a finite-dimensional vector space, we have a Hilbert space of functions $f(x)$, and operators acting on functions via integration with the kernel.

Numerically, Fredholm equations of the first kind are **very** ill-conditioned.

On an intuitive level, a convolution with a smooth kernel is a smoothing operation --- and it is designed to *lose* information, e.g. about fast oscillations in $f(x)$.

Naive approaches at solving a FE of the first kind are prone to the occurence of the see-saw instability, where noise grows to dominate the solution. 

In general, to solve a FE of the first kind, one needs to use some prior information about $f(x)$ or impose some regularization.

Note however, that FEs of the first kind with *signular* kernels can actually be *better* defined. For example, consider a (highly singular) kernel $K(x, t) = \delta(x - t)$, where $\delta(x)$ is the Dirac delta function: the solution is just $f(x) \equiv g(x)$. 

## Fredholm equations of the second kind

These FEs contain the unknown function both inside and outside of the integral:

$$
\lambda \int_a^b\! K(x, t)\, \color{blue}{f(t)} \, dt + g(x) = \color{blue}{f(x)}
$$

Here $\lambda$ is a real number. With $g(x) = 0$ for $x\in [a, b]$, the equation is called *homogenous* (однородное), otherwise it's *inhomogenous* .

If $K(x, t)$ is bounded and square-integrable on $[a, b] \times [a, b]$, the homogenous equation

$$
\lambda \int_a^b\! K(x, t)\, \color{blue}{f(t)} \, dt = \color{blue}{f(x)}
$$

has a finite or a countable set (счетное число) of solutions for $\lambda$ equal to *characteristic values*  $\lambda_n$ with $n=0, 1, \dots$. The inverse values $\sigma_n = 1/\lambda_n$ are then *eigenvalues* and the corresponding solutions $f_n(x)$ are *eigenfunctions* (собственные значения и собственные функции).

Again, notice the similarity to the finite-dimensional eigenproblem $\mathbf{A}\, \mathbf{x} = \sigma\, \mathbf{x}$.

Generally FEs of the second kind are much more stable than FEs of the first kind.

Loosely speaking, *FE of the second kind is typically well conditioned unless $\lambda$ is small.*
Integration maps rapidly oscillating functions to near zero, hence a FE can be expected to have many *"large"* eigenvalues, which we are not very interested in and which do not pose much trouble for numerics.

Putting it differently, for $\lambda \ll 1$ FE of the second kind is "close" to an FE of the first kind, so we can expect the stability to be progressively worse for smaller values $\lambda$.

*Fredholm alternative*: If $\lambda = \lambda_n$, a homogenous FE of the second kind has a solution; an inhomogenous FE does not. 

For analytical methods of solving FEs of the second kind, see e.g. 

1. В.А. Попов, Сборник задач по интегральным уравнениям, КазГУ, 2006.

For introduction into numerical methods see e.g.

2. Numerical Recipes 3rd ed, Chapter 19 (Chapter 18 in earlier editions)

3. L.M. Delves and J.L. Mohamed, Computational methods for intergral equations, CUP, 1988.

## Numeric solutions of FE of the second kind

The basic idea is to approximate the integral by some quadrature formula

$$
\int_a^b \! \xi(t)\, dt \approx \sum_j w_j\, \xi(t_j)
$$

with appropriate weights $w_j$ and nodes $t_j$ and $j=1, \dots, N$. The accuracy of the approximation is controlled by $N$.

This way, the FE is approximated by 

$$
\lambda \sum_j w_j\, K(x, t_j)\, \color{blue}{f(t_j)} + g(x) = \color{blue}{f(x)}
$$

Note that here $x$ is a continuous variable, and we only discretized $t$.


Evaluating this equation on the grid $x = t_k$, we obtain

$$
\lambda \sum_j w_j \, K_{k, j}\, \color{blue}{f_j} + g_k = \color{blue}{f_k}
$$

where $f_j \equiv f(t_j)$, $g_j \equiv g(t_j)$ and $K_{k, j} \equiv K(t_k, t_j)$. This is nothing but a system of linear algebraic equations for the vector of $\color{blue}{f_j}$.

Its solution gives the values of the unknown function $f(x)$ at the discrete values $x=t_j$ for $j=1, \dots, N$.

For $x \neq t_j$, one should *not* use interpolation because it destroys accuracy. Instead, one uses the original approximation with continuous $x$ above.

## How to choose nodes and weights.

The size of the algebraic system is $N$. The computational complexity is thus $O(N^3)$. Gaussian quadratures are thus preferable to Newton-Cotes formulas.


Ideally, one would use $K(x, t)$ as the weight function for the quadrature. However, computations get messy because nodes and weights depend on $x$. 

If the kernel has a multiplicative piece, try using it as a weight function. E.g., for $K = \cos{(xt)} / \sqrt{1 - t^2}$ use Chebyshev polynomials (where $w(t) = 1 / \sqrt{1-t^2}$).

Otherwise, a Gauss-Legendre quadrature ($w(x) = 1$) is a good default choice.

## Singular kernels

Sometimes you can change variables to regularize the integral or use the weight function with the same singularity.

#### Subtraction of singularity.

A common case is the convolution type integral, where the kernel is singular at $x = t$. In that case, the Nystrom method discussed above fails, because it needs to evaluate $K_{j, j} \equiv K(t_j, t_j)$.

A standard trick is to *subtract the singularity*:

\begin{eqnarray*}
\int_a^b \! K(x, t) \, f(t) \, dt &=
\int_a^b \! K(x, t) \bigl[\,f(t) - f(x) + f(x) \bigr] \, dt \\&= 
\int_a^b \! K(x, t) \bigl[\,f(t) - f(x) \bigr] \, dt + f(x) \int_a^b \! K(x, t) \, dt \\
\end{eqnarray*}

This way, replacing the first integral by the quadrature formula, we obtain

$$
\sum_j \, w_j K_{k, j} \bigl(f_j - f_k\bigr)
$$

so that the diagonal terms with $k = j$ disappear from the resulting system of algebraic equations.

#### Method of moments.

Consider a uniform mesh $\{t_j\}$ on the interval $(a, b)$. Recall the construction of Newton-Cotes quadratures: we adjust the weights $\{ w_j\}$ such that a $k$-point elementary formula integrates polynomials up to degree $k$ *exactly*.

As a direct generalization, we can construct a $k$-point formula which integrates a polynomial *times $K(x, t)$* exactly. For this, we need to be able to calculate analytically or numerically

$$
\mu_q(x) = \int_a^b \! t^q \, K(x, t) \, dt
$$

for all $x \in (a, b)$ and $q = 0, \dots k$.

Then, we replace the integration with the quadrature formula where the coefficients depend on $\mu_q(t_j)$ and obtain a system of linear algebraic equations.